<a href="https://colab.research.google.com/github/Dannatinjaca/PROGCOM-A/blob/main/juego_halloween.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
from abc import ABC, abstractmethod
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

class Personaje:
    def __init__(self, nombre):
        self.nombre = nombre
        self.vida = 5
        self.dulces = 0
        self.linterna = False
        self.usos_linterna = 0

    def mostrar_estado(self):
        return widgets.HTML(f"""
        <h3>🎃 Estado de {self.nombre}</h3>
        <ul>
            <li>❤️ Vida: {self.vida}</li>
            <li>🍬 Dulces: {self.dulces}</li>
            <li>🔦 Linterna: {'Sí (' + str(self.usos_linterna) + ' usos)' if self.linterna else 'No'}</li>
        </ul>
        """)

class Enemigo(ABC):
    def __init__(self, nombre):
        self.nombre = nombre

    @abstractmethod
    def atacar(self, jugador):
        pass

class Fantasma(Enemigo):
    def __init__(self):
        super().__init__("Fantasma 👻")

    def atacar(self, jugador):
        jugador.vida -= 1
        return "El fantasma te atraviesa sin piedad. Sientes un escalofrío... Perdiste 1 de vida."

class Zombi(Enemigo):
    def __init__(self):
        super().__init__("Zombi 📛")

    def atacar(self, jugador):
        if random.choice([True, False]):
            return "¡Lograste esquivar la mordida del zombi a último momento!"
        else:
            jugador.vida -= 1
            return "El zombi te alcanzó y te mordió. Perdiste 1 de vida."

class Juego:
    META_DULCES = 30

    def __init__(self):
        self.output_area = widgets.VBox()
        self.instrucciones()

    def instrucciones(self):
        intro_out = widgets.Output()
        with intro_out:
            clear_output()
            display(widgets.HTML(f"""
            <h2>🎃 Bienvenido a <i>Dulces o Susto</i> 🎃</h2>
            <p><b>Reglas del juego:</b></p>
            <ul>
                <li>Tu objetivo es recolectar <b>{self.META_DULCES} dulces</b> 🎯</li>
                <li>Si tu <b>vida</b> baja a 0, pierdes ☠️</li>
                <li>Decide entre <b>correr</b>, <b>defenderte</b> o <b>usar linterna</b></li>
                <li><b>Linterna:</b> sirve para espantar enemigos. Solo tiene 2 usos 🔦</li>
                <li><b>Botiquín:</b> recuperas 1 punto de vida si encuentras uno 🩹</li>
            </ul>
            """))
            nombre_input = widgets.Text(description='Nombre:', placeholder='Ingresa tu nombre')
            comenzar_btn = widgets.Button(description='Comenzar', button_style='success')

            def comenzar(_):
                if nombre_input.value.strip():
                    self.turno_actual = 0
                    self.jugador = Personaje(nombre_input.value.strip())
                    self.output_area.children = ()
                    self.jugar()

            comenzar_btn.on_click(comenzar)
            display(nombre_input, comenzar_btn)

        self.output_area.children = (intro_out,)

    def jugar(self):
        self.turno_actual += 1
        evento_out = widgets.Output()
        botones_out = widgets.Output()
        area = widgets.VBox()
        area.children = (widgets.HTML(f"<h3>📍 Turno {self.turno_actual}</h3>"), self.jugador.mostrar_estado(), evento_out, botones_out)
        self.output_area.children += (area,)
        self.generar_evento(evento_out, botones_out)

    def generar_evento(self, evento_out, botones_out):
        opciones = ["dulces", "fantasma", "zombi", "nada", "linterna", "botiquin"]
        probabilidades = [0.22, 0.22, 0.22, 0.1, 0.08, 0.1]
        evento = random.choices(opciones, weights=probabilidades, k=1)[0]

        if evento == "dulces":
            with evento_out:
                with open("dulce.png", 'rb') as f:
                    img_data = f.read()
                display(widgets.Image(value=img_data, format='png', width=150))
                display(widgets.HTML("<p>🍬 ¡Has encontrado un dulce! ¿Quieres abrir la puerta?</p>"))
            self.opciones_dulces(botones_out, evento_out)

        elif evento == "linterna":
            with evento_out:
                with open("linterna.png", 'rb') as f:
                    img_data = f.read()
                display(widgets.Image(value=img_data, format='png', width=150))
                if not self.jugador.linterna:
                    self.jugador.linterna = True
                    self.jugador.usos_linterna = 2
                    display(widgets.HTML("<p>🔦 Encontraste una linterna mágica. Puedes usarla 2 veces para espantar enemigos.</p>"))
                else:
                    display(widgets.HTML("<p>🔦 Encontraste otra linterna, pero ya tienes una.</p>"))
            self.boton_siguiente_turno(botones_out)

        elif evento == "botiquin":
            with evento_out:
                with open("botiquin.png", 'rb') as f:
                    img_data = f.read()
                display(widgets.Image(value=img_data, format='png', width=150))
                self.jugador.vida += 1
                display(widgets.HTML("<p>🩹 Has encontrado un botiquín y ganaste +1 de vida.</p>"))
            self.boton_siguiente_turno(botones_out)

        elif evento == "nada":
            with evento_out:
                display(widgets.HTML("<p>🌫️ La calle está vacía y silenciosa...</p>"))
            self.boton_siguiente_turno(botones_out)

        else:
            self.enemigo = Fantasma() if evento == "fantasma" else Zombi()
            with evento_out:
                img_path = "fantasma.png" if evento == "fantasma" else "zombie.png"
                with open(img_path, 'rb') as f:
                    img_data = f.read()
                display(widgets.Image(value=img_data, format='png', width=150))
                display(widgets.HTML(f"<p><b>¡Un {self.enemigo.nombre} aparece de repente!</b></p>"))
            self.opciones_enemigo(botones_out, evento_out)

    def opciones_dulces(self, botones_out, evento_out):
        with botones_out:
            btn_abrir = widgets.Button(description='🚪 Abrir puerta', button_style='primary')

            def abrir_puerta(_):
                cantidad = random.randint(1, 3)
                self.jugador.dulces += cantidad
                with evento_out:
                    display(widgets.HTML(f"<p>🏠 Te dieron <b>{cantidad}</b> dulces. ¡Qué suerte!</p>"))
                self.boton_siguiente_turno(botones_out)

            btn_abrir.on_click(abrir_puerta)
            display(btn_abrir)

    def opciones_enemigo(self, botones_out, evento_out):
        with botones_out:
            btn_correr = widgets.Button(description='🏃 Correr')
            btn_defender = widgets.Button(description='🛡️ Defenderse')
            btn_linterna = widgets.Button(description='🔦 Usar linterna')

            def limpiar_y_avanzar(extra_accion=None):
                botones_out.clear_output()
                if extra_accion:
                    with evento_out:
                        display(widgets.HTML(f"<p>{extra_accion}</p>"))
                self.boton_siguiente_turno(botones_out)

            def accion_correr(_):
                with evento_out:
                    if random.choice([True, False]):
                        limpiar_y_avanzar("Corriste y escapaste.")
                    else:
                        resultado = self.enemigo.atacar(self.jugador)
                        limpiar_y_avanzar("Te tropezaste y el enemigo te alcanzó.<br>" + resultado)

            def accion_defender(_):
                with evento_out:
                    if isinstance(self.enemigo, Fantasma):
                        resultado = self.enemigo.atacar(self.jugador)
                        limpiar_y_avanzar("El fantasma atraviesa tu defensa.<br>" + resultado)
                    else:
                        if random.choice([True, False]):
                            limpiar_y_avanzar("Lograste defenderte del zombi.")
                        else:
                            resultado = self.enemigo.atacar(self.jugador)
                            limpiar_y_avanzar("Fallaste al defenderte del zombi.<br>" + resultado)

            def accion_linterna(_):
                with evento_out:
                    if self.jugador.linterna and self.jugador.usos_linterna > 0:
                        self.jugador.usos_linterna -= 1
                        mensaje = "La linterna ahuyenta al enemigo."
                        if self.jugador.usos_linterna == 0:
                            self.jugador.linterna = False
                            mensaje += " Ya no te quedan usos."
                        limpiar_y_avanzar(mensaje)
                    else:
                        resultado = self.enemigo.atacar(self.jugador)
                        limpiar_y_avanzar("No tienes linterna. El enemigo te ataca.<br>" + resultado)

            btn_correr.on_click(accion_correr)
            btn_defender.on_click(accion_defender)
            btn_linterna.on_click(accion_linterna)
            display(widgets.HBox([btn_correr, btn_defender, btn_linterna]))

    def boton_siguiente_turno(self, botones_out):
        with botones_out:
            siguiente_btn = widgets.Button(description='🕐 Siguiente turno', button_style='primary')

            def avanzar(_):
                self.comprobar_estado()

            siguiente_btn.on_click(avanzar)
            display(siguiente_btn)

    def comprobar_estado(self):
        if self.jugador.vida <= 0:
            final = widgets.Output()
            with final:
                display(widgets.HTML("<h2 style='color:red'>💀 Perdiste... los monstruos fueron más rápidos que tú.</h2>"))
                self.boton_reiniciar(final)
            self.output_area.children += (final,)

        elif self.jugador.dulces >= self.META_DULCES:
            final = widgets.Output()
            with final:
                display(widgets.HTML("<h2 style='color:green'>🎉 ¡Ganaste! Has recolectado todos los dulces que querías.</h2>"))
                self.boton_reiniciar(final)
            self.output_area.children += (final,)

        else:
            self.jugar()

    def boton_reiniciar(self, area):
        reiniciar_btn = widgets.Button(description='🔁 Reiniciar Juego', button_style='warning')

        def reiniciar(_):
            self.instrucciones()

        reiniciar_btn.on_click(reiniciar)
        with area:
            display(reiniciar_btn)

juego = Juego()
display(juego.output_area)
